In [1]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=030be72365305d0067b687cf269c136e2124977b07f078327a5dea942c075041
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=TNNZwkOuGnSNjxY39UAyGwyTZUtWJ7&prompt=consent&token_usage=remote&access_type=offline&code_challenge=pXdHU-PT6qWoMPyY-MeIcmmvGHWf4VGljWVLB0uMVQ4&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrCOUpj0yFYyEdC6EDe6vPlCQfZTtN-Marfl6jB0X5oP8DOLjI6bbGHJUwZNdftGGw

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [5]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [6]:
client = bigquery.Client(project='repositoriodedadosgpsp')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
# Definir a consulta SQL
query = """
SELECT *
FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_vinculos_publicos_v2`
"""

# Executar a consulta e salvar os resultados em um DataFrame do pandas
df = client.query(query).to_dataframe()

# Exibir os primeiros registros do DataFrame
print(df.head())

    ano variavel         categoria  media_remuneracao
0  2022   genero  Não identificado           0.000000
1  2022   genero          Feminino        5099.993482
2  2017   genero          Feminino        3791.736833
3  2009   genero          Feminino        1887.628480
4  2008   genero          Feminino        1723.506923


In [8]:
df['ano'].unique()

## tenho de 2004 a 2021:



<IntegerArray>
[2022, 2017, 2009, 2008, 2013, 2019, 2007, 2010, 2018, 2016, 2020, 2011, 2015,
 2003, 2006, 2005, 2012, 2014, 2004, 2021]
Length: 20, dtype: Int64

In [9]:
# Definir a consulta SQL
query = """
SELECT * FROM `repositoriodedadosgpsp.Datalake.ipca`
"""



# Executar a consulta e salvar os resultados em um DataFrame do pandas
ipca = client.query(query).to_dataframe()

# Exibir os primeiros registros do DataFrame
print(ipca.head())

    ano  acumulado_dezembro_ano           it  fator_correcao
0  2018                    3.75  4098.727240        1.328049
1  2011                    6.50  2734.909768        1.990308
2  2020                    4.52  4468.629667        1.218117
3  2006                    3.14  2101.282770        2.590471
4  1996                    9.56  1095.600000        4.968339


In [10]:
ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ano                     29 non-null     Int64  
 1   acumulado_dezembro_ano  29 non-null     float64
 2   it                      29 non-null     float64
 3   fator_correcao          29 non-null     float64
dtypes: Int64(1), float64(3)
memory usage: 1.1 KB


In [11]:
df1= df.merge(ipca[['ano','fator_correcao']], right_on='ano',left_on='ano')

In [12]:
df1['media_remuneracao_ajustada']= df1['media_remuneracao']* df1['fator_correcao']

In [13]:
df1

,ano,variavel,categoria,media_remuneracao,fator_correcao,media_remuneracao_ajustada
0,2022,genero,Não identificado,0.000000,1.046200,0.000000
1,2022,genero,Feminino,5099.993482,1.046200,5335.613181
2,2022,genero,Masculino,7201.277814,1.046200,7533.976849
3,2022,poderes,Outros,8540.517166,1.046200,8935.089059
4,2022,poderes,Executivo,4486.838805,1.046200,4694.130758
...,...,...,...,...,...,...
491,2021,grau_instrucao,Até Mestrado,8519.270156,1.106775,9428.915056
492,2021,grau_instrucao,Até Doutorado,12202.513889,1.106775,13505.437065
493,2021,grau_instrucao,Até Fundamental,2348.324320,1.106775,2599.066602
494,2021,grau_instrucao,Até Ensino Médio,3094.496974,1.106775,3424.911827


In [14]:
df1 = df1[['ano', 'variavel', 'categoria', 'media_remuneracao',
       'fator_correcao', 'media_remuneracao_ajustada']]

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ano                         496 non-null    Int64  
 1   variavel                    496 non-null    object 
 2   categoria                   496 non-null    object 
 3   media_remuneracao           496 non-null    float64
 4   fator_correcao              496 non-null    float64
 5   media_remuneracao_ajustada  496 non-null    float64
dtypes: Int64(1), float64(3), object(2)
memory usage: 23.9+ KB


In [16]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência'),
 bigquery.SchemaField('variavel','STRING',description='De qual variável aquela remuneração se refere'),
 bigquery.SchemaField('categoria','STRING',description='Categoria dentro daquela variável'),
 bigquery.SchemaField('media_remuneracao','FLOAT',description='Valor nominal da média de remuneração daquele ano'),
bigquery.SchemaField('fator_correcao','FLOAT',description='índice para correção monetária baseando-se no IPCA para dezembro de 2022'),
bigquery.SchemaField('media_remuneracao_ajustada','FLOAT',description='Valor nominal da média de remuneração daquele ano ajustado para dezembro de 2022')
 ]


In [17]:
client = bigquery.Client(project='repositoriodedadosgpsp')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [18]:
dataset_ref = client.dataset('perfil_remuneracao')

In [19]:
table_ref = dataset_ref.table('RAIS_remuneracao_vinculos_publicos_v_3')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=c5fc3193-fc3b-430c-b937-c62402565d54>